In [19]:
from pre_process.load_sleep_data import LoadSleepData
from pre_process.pre_process import PreProcess
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
import sys
from sklearn.manifold import TSNE
from collections import Counter
import tensorflow as tf
from tensorboard.plugins import projector
from nn.losses import EDLLoss
print(f"curernt dir: {os.getcwd()}")

curernt dir: c:\Users\takadamalab\git\sleep_study


In [20]:
tf.keras.backend.set_floatx("float32")
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [21]:
load_sleep_data = LoadSleepData(data_type="spectrum")
pre_process = PreProcess(load_sleep_data)
datas = load_sleep_data.load_data(load_all=True)
(train, test) = pre_process.split_train_test_from_records(datas,
                                                          test_id=0,
                                                          pse_data=False)
(x_train, y_train), (x_test, y_test) = pre_process.make_dataset(train=train, 
                                                      test=test,
                                                      is_storchastic=False,
                                                      to_one_hot_vector=False,
                                                      pse_data=False,
                                                      is_shuffle=False)

*** すべての被験者を読み込みます（load_dataの引数:nameは無視します） ***
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Li_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Murakami_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Yamamoto_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Kumazawa_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Hayashi_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Kumazawa_F_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Takadama_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Hiromoto_20210320-011750.savを読み込んでいます...
C:\Users\takadamalab\sleep_study\datas\pre_processed_data\H_Kashiwazaki_20210320-011750.savを読み込んでいます...
訓練データのサイズを揃えます
訓練データの各睡眠段階（補正前） Counter({2: 1

In [22]:
x_train.shape

(5000, 512, 1)

In [23]:
# モデルを25次元の空間に出力
date_id = "20210608-144244"
path = os.path.join(os.environ["sleep"], "models", "H_Li", date_id)
model = tf.keras.models.load_model(path, custom_objects={"EDLLoss":EDLLoss(K=5,annealing=0.1)})

In [24]:
hidden_layer_id = -1
new_model = tf.keras.Model(inputs=model.input, outputs=model.layers[hidden_layer_id].output)
# new_model.summary()
# new_model.compile()
result = new_model.predict(x_test, batch_size=32)

In [25]:
# log先の設定
log_dir = os.path.join(os.environ["sleep"], "logs", "H_Li", "spectrum", f"layer_{hidden_layer_id}", "test")
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
# ラベルの保存
label_path = os.path.join(log_dir, "metadata.tsv")
with open(label_path, "w") as f:
    f.write("index\tlabel\n")
    for index, label in enumerate(y_test):
        f.write(f"{index}\t{str(label)}\n")

In [26]:
# チェックポイント作成の続き（上のpredictの計算が長いので，セルを分ける）
embedding_var = tf.Variable(result, name="dense")
check_point_file = os.path.join(log_dir, "embedding.ckpt")
ckpt = tf.train.Checkpoint(embedding=embedding_var)
ckpt.save(check_point_file)

'C:\\Users\\takadamalab\\sleep_study\\logs\\H_Li\\spectrum\\layer_-1\\test\\embedding.ckpt-1'

In [27]:
# projectorの設定
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = "metadata.tsv"
projector.visualize_embeddings(log_dir, config)